In [13]:
import numpy as np
from scipy.stats import f, chi2

# Дисперсионный анализ
Дисперсионный анализ — метод в математической статистике, направленный на поиск зависимостей в экспериментальных данных путём исследования значимости различий в средних значениях.

## Однофакторный дисперсионный анализ
Пусть на значения зависимой переменной влияет только один фактор, он может принимать k >= 2 значений. Будем считать, что дисперсии выборок одинаковы, а матожидание необходимо найти.
Посчитаем следующие значения:
$$Q_1 = \sum_{j=1}^k\sum_{i=1}^{n_j}{(x_{ij} - \bar{x_j})^2}$$, где $n_j$ - длина выборки j, $\bar{x_j}$ - среднее значение выборки j, а всего есть k выборок (могут быть разной длины).
$$Q_2 = \sum_{j=1}^k{n_j (\bar{x_j} - \bar{x})^2}$$, где $\bar{x}$ - общее среднее всех выборок, а $n_j$ - длина соответствующей выборки j.

Тогда $Q_1$/(N-k) равно точечной оценке дисперсий всех выборок (они равны) (внутри группы), а $Q_2$ - оценкой дисперсии, но только в случае, если выполняется нулевая гипотеза.  

Тогда составим следующую статистику: $$F = \frac{Q_2 / (k-1)}{Q_1 / (N-k)}$$, где k и N - количество выборок и общее количество элементов в них (суммарно) соответственно, а $Q_1$ и $Q_2$ - статистики, вычисленные на предыдущем шаге.  
Такая статистика будет иметь распределение Фишера с (k-1, N-k) степенями свободы, если выполняется нулевая гипотеза, а в случае невыполнения нулевой гипотезы, ее значение будет возрастать. Следовательно, критическая область будет равна следующему интервалу: $$V_k = (u_{1-\alpha, k-1, N-k}; +\infty)$$, а p-value = $1 - F_{k-1, N-k}(F_В)$, где $F_В$ - выборочное значение статистики F.  

Если выборочное значение статистики попадает в критическую область (или p-value меньше заданной величины), то считаем, что фактор влияет на матожидание зависимой переменной.

#### Пример
Время химической реакции при различном содержании катализатора распределилось следующим образом:
* 05%: 5.9, 6.0, 7.0, 6.5, 5.5, 7.0, 8.1, 7.5, 6.2, 6.4, 7.1, 6.9
* 10%: 4.0, 5.1, 6.2, 5.3, 4.5, 4.4, 5.3, 5.4, 5.6, 5.2
* 15%: 8.2, 6.8, 8.0, 7.5, 7.2, 7.9, 8.1, 8.5, 7.8, 8.1

Выборки получены из нормально распределенных генеральных совокупностей с равными дисперсиями. Проверить гипотезу о равенстве средних. Уровень значимости принять равным 0.1.

In [14]:
alpha = 0.1

a1 = np.array([5.9, 6.0, 7.0, 6.5, 5.5, 7.0, 8.1, 7.5, 6.2, 6.4, 7.1, 6.9])
a2 = np.array([4.0, 5.1, 6.2, 5.3, 4.5, 4.4, 5.3, 5.4, 5.6, 5.2])
a3 = np.array([8.2, 6.8, 8.0, 7.5, 7.2, 7.9, 8.1, 8.5, 7.8, 8.1])

# вычисляем общую выборку, общую длину и общее среднее
k = 3
A = np.concatenate([a1, a2, a3])
N = len(A)
m = A.mean()

# задаем распределение Фишера, которое будет использоваться
f_local = f(k - 1, N - k)
print(f"quantile with alpha = 0.1 -- {f_local.ppf(1- 0.1)}")

a1_d = sum((a1 - a1.mean())**2)
a2_d = sum((a2 - a2.mean())**2)
a3_d = sum((a3 - a3.mean())**2)

Q1 = a1_d + a2_d + a3_d
print(f"Q1 = {Q1}")
sigma1= Q1/(N-k)
print(f"sigma = Q1/(N-k)  -- {sigma1}")

Q2 = \
    len(a1) * (m - a1.mean())**2 + \
    len(a2) * (m - a2.mean())**2 + \
    len(a3) * (m - a3.mean())**2
print(f"Q2 = {Q2}")

Fv = (Q2 / (k - 1)) / (Q1 / (N - k))
print(f"Выборочное значение статистики: {Fv}")
print(f"Критическая область: ({f_local.ppf(1-alpha)}; +inf)")
print(f"p-value: {1 - f_local.cdf(Fv)}")

quantile with alpha = 0.1 -- 2.4954833142354644
Q1 = 11.951499999999998
sigma = Q1/(N-k)  -- 0.41212068965517235
Q2 = 37.08349999999997
Выборочное значение статистики: 44.991068066769834
Критическая область: (2.4954833142354644; +inf)
p-value: 1.2890964962153362e-09


### Метод линейных контрастов
Метод линейных контрастов позволяет узнать, матожидание каких именно генеральных совокупностей различаются (если однофакторный анализ показал, что они различны).


Линейный контраст - линейная комбинация
$$L_k = \sum_{j=1}^k{c_j a_j}$$

где  $c_j, j = 1,...,k: \sum_{j=1}^k{c_j} = 0$

Точечная оценка линейного контраста: 

$$\hat{L}_k = \sum_{j=1}^k{c_j \bar{x}_j}$$
Дисперсия: 

$$D(\hat{L}_k) = \sum_{j=1}^k{c_j^2 D(\bar{x}_j)} = \sum_{j=1}^k{\frac{c_j^2}{n_j} \sigma^2}$$

Т.к. дисперсия не известна найдем оценку дисперсии
$$S_{\hat{L}_k}^2 = \sum_{j=1}^k{c_j^2 D(\bar{x}_j)} = \sum_{j=1}^k{\frac{c_j^2}{n_j} \frac{Q_1}{N-k}}$$

#### Лемма (Метод Шеффе)

для любой совокупности векторов $c_1, ... , c_k: \sum_{j=1}^k{c_j} = 0$
вероятность одновременного выполнения неравенств 
$$\left| \sum_{j=1}^k{c_j ( a_j - \bar{x}_j)}  \right| < S_{\hat{L}_k} \sqrt{(k-1)u_{1-\alpha, k-1, N-k}}$$
не меньше 1 - $\alpha$  (распределение Фишера)

#### Проверка гипотезы

Тогда для проверки гипотезы:
$$H_0 : L_k = 0$$
$$H_1 : L_k \neq 0$$


воспользуемся интервалом из  метода Шеффе:

$$\left( \hat{L}_k - S_{\hat{L}_k} \sqrt{(k-1)u_{1-\alpha, k-1, N-k}} , 
\hat{L}_k +  S_{\hat{L}_k} \sqrt{(k-1)u_{1-\alpha, k-1, N-k}} \right)$$

если  0 $\in$ данному интервалу то принимаем 0 гипотезу , в противном случае отвергаем в пользу первой

Найдите точечную оценку линейного контраста для проверки частной нулевой гипотезы о равенстве среднего времени химической реакции при содержании катализатора  5% и 10% (H0:a5%=a10%)

In [15]:
alpha = 0.1

#5%
a1 = np.array([5.9, 6.0, 7.0, 6.5, 5.5, 7.0, 8.1, 7.5, 6.2, 6.4, 7.1, 6.9])
#10%
a2 = np.array([4.0, 5.1, 6.2, 5.3, 4.5, 4.4, 5.3, 5.4, 5.6, 5.2])
#15%
a3 = np.array([8.2, 6.8, 8.0, 7.5, 7.2, 7.9, 8.1, 8.5, 7.8, 8.1])

# вычисляем общую выборку, общую длину и общее среднее
k = 3
A = np.concatenate([a1, a2, a3])
N = len(A)
m = A.mean()

L_k12 = 1*(a1.mean()) -1*(a2.mean())

print(f"Lk for 1 and 2 sets  -- {L_k12}")




Lk for 1 and 2 sets  -- 1.5750000000000002


Найдите доверительный интервал для линейного контраста для проверки частной нулевой гипотезы о равенстве среднего времени химической реакции при содержании катализатора  5% и 10% (H0:a5%=a10%) и примите статистическое решение о равенстве сравниваемых мат.ожиданий

In [16]:
# задаем распределение Фишера, которое будет использоваться
f_local = f(k - 1, N - k)
print(f"quantile with alpha = 0.1 -- {f_local.ppf(1- 0.1)}")

a1_d = sum((a1 - a1.mean())**2)
a2_d = sum((a2 - a2.mean())**2)
a3_d = sum((a3 - a3.mean())**2)

Q1 = a1_d + a2_d + a3_d
print(f"Q1 = {Q1}")
sigma1= Q1/(N-k)
print(f"sigma = Q1/(N-k)  -- {sigma1}")

S2 = sigma1 * (1/len(a1) + 1/len(a2))
delta = np.sqrt(S2)* np.sqrt((k-1)*f_local.ppf(1-alpha))

left = L_k12 - delta
right = L_k12 + delta
Fv = (Q2 / (k - 1)) / (Q1 / (N - k))
print(f"интервал: ({left}, {right})")
print(f"решение: {(left <= 0) & (right >=0)}")

quantile with alpha = 0.1 -- 2.4954833142354644
Q1 = 11.951499999999998
sigma = Q1/(N-k)  -- 0.41212068965517235
интервал: (0.9609195669039591, 2.1890804330960414)
решение: False


### Критерий Краскела-Уоллиса

Объединяем все выборки в одну большую.

Составляем вариационный ряд

$X_{(1)} <= X_{(2)} <= ... <= X_{(N)}$

$ R_{ij} =  rank(X_{ij})$

$ \bar{R_j} = \frac{1}{n_j} \sum_{i=1}^{n_{j}}{R_{ij}}$

$ \bar{R} = \frac{1}{N} \sum_{j=1}^{k} {\sum_{i=1}^{n_{j}}{R_{ij}}} = \frac{N+1}{2}$


В качестве статистики берем 
$$ Q_{2} = \sum_{j=1}^{k}{(\bar{R_j} - \bar{R})^2}$$

На практике же используют статистику H которая дает оценку для Q_2 

$$H = \frac{12}{N(N+1)}Q_2 = \frac{12}{N(N+1)} \left(  \sum_{j=1}^{k}{n_j\bar{R_j^2}} \right) - 3(N+1)$$

В случаях : 

k = 3 и $n_j >= 5$

k > 3 и $n_j >= 4$

используют приближенное значение статистики Н :  $\chi^2(k-1)$
иначе - используют таблицы

тогда критическая область: 

$$V_k = (\chi^2_{1-\alpha}(k-1), +\infty)$$ 

если попали в область - отклоняем $H_0$

Если есть одинаковые значения из разных выборок, то существует статистика $H^*$ - скорректированная

А так же можно исопльзовать аппроксимацию Имана-Давенпорта  при больших N (Стастистика J)

### Пример
Проверьте, правда ли, что болевой барьер у блондинов и брюнетов разный.

Цвет волос	Болевой барьер

Светлый блондин: 62, 60, 71, 55, 48

Темный блондин: 63, 57, 52, 41, 43

Светлый брюнет: 42, 50, 44, 37

Темный брюнет: 32, 39, 51, 30, 35


Найдите средний ранг болевого барьера для  светлых брюнетов.
Формат ответа: значение среднего ранга с точностью 2 знака после запятой

In [17]:
a1 = np.array([62, 60, 71, 55, 48])
a2 = np.array([63, 57, 52, 41, 43])
a3 = np.array([42, 50, 44, 37])
a4 = np.array([32, 39, 51, 30, 35])

a1 = [(x, 1) for x in a1]
a2 = [(x, 2) for x in a2]
a3 = [(x, 3) for x in a3]
a4 = [(x, 4) for x in a4]
# вычисляем общую выборку, общую длину и общее среднее
k = 4
A = np.concatenate([a1, a2, a3,a4])
N = len(A)
A
A_sorted = sorted(A,key=lambda x: x[0])

A_ranked = [(x[0],x[1], i+1) for i,x in enumerate(A_sorted)]
print(A_ranked)
R3 = sum(x[2] for x in A_ranked if x[1] == 3)/ len(a3)

print(f"mean rank for a3 : {R3}")


[(30, 4, 1), (32, 4, 2), (35, 4, 3), (37, 3, 4), (39, 4, 5), (41, 2, 6), (42, 3, 7), (43, 2, 8), (44, 3, 9), (48, 1, 10), (50, 3, 11), (51, 4, 12), (52, 2, 13), (55, 1, 14), (57, 2, 15), (60, 1, 16), (62, 1, 17), (63, 2, 18), (71, 1, 19)]
mean rank for a3 : 7.75



Найдите выборочное значение статистики критерия и примите статистическое решение. Уровень значимости 0.05

In [ ]:
R1 = sum(x[2] for x in A_ranked if x[1] == 1)/ len(a1)
R2 = sum(x[2] for x in A_ranked if x[1] == 2)/ len(a2)
R3 = sum(x[2] for x in A_ranked if x[1] == 3)/ len(a3)
R4 = sum(x[2] for x in A_ranked if x[1] == 4)/ len(a4)
alpha = 0.05
lens = [len(a1), len(a2), len(a3), len(a4)]
R_i = [R1, R2, R3, R4]

sum_R  = np.sum([r*r*nj for r,nj in zip(R_i, lens)])
# задаем распределение Chi2, которое будет использоваться
chi2_local = chi2(k-1)
H = 12/(N*(N+1)) * sum_R - 3*(N+1)
print(f"Критическая область: ({chi2_local.ppf(1-alpha)}; +inf)")
print(f"Выборочное значение  : {H}")

### Еще примеры


Сравните среднюю продолжительность успешных проходов в 10 матчах по рэгби.

Уровень значимости 0.05. Предполагаем, что продолжительность успешных проходов имеет нормальное распределение.

Найдите статистику критерия и примите статистическое решение



In [19]:
import pandas as pd
df = pd.read_csv("rugby.txt",",")  #второй аргумент задает разделитель

In [44]:
[x for x in range(1,11)]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [46]:
variances

[21443.353195876283,
 25753.2155140187,
 12906.731182795704,
 22006.80224719101,
 23884.31186813188,
 19489.173100000007,
 21070.415180722888,
 16022.388518518514,
 14298.688785046726,
 14365.913846153842]

In [50]:
k = len(df['Game'].unique())
N = len(df)
m = df['Time'].mean()
alpha = 0.05
f_local = f(k - 1, N - k)
print(f"quantile -- {f_local.ppf(1-alpha)}")


variances = [ sum((df[df['Game'] == x]['Time'] - df[df['Game'] == x]['Time'].mean())**2) for x in range(1, 11)]
lens = [len(df[df['Game'] == x]['Time']) for x in range(1,11)]
Q1 = sum(variances)
print(f"Q1 = {Q1}")
sigma1= Q1/(N-k)
print(f"sigma = Q1/(N-k)  -- {sigma1}")

means = [df[df['Game'] == x]['Time'].mean() for x in range(1,11)]
Q2_list = [l*(m-mm)**2 for l,mm in zip(lens, means)]
print(Q2_list)
Q2 = sum(Q2_list)
print(f"Q2 = {Q2}")

Fv = (Q2 / (k - 1)) / (Q1 / (N - k))
print(f"Выборочное значение статистики: {Fv}")
print(f"Критическая область: ({f_local.ppf(1-alpha)}; +inf)")
print(f"p-value: {1 - f_local.cdf(Fv)}")

quantile -- 1.889525776660891
Q1 = 191240.99343845557
sigma = Q1/(N-k)  -- 197.3591263554753
[944.782097908087, 351.86278152583895, 749.961098231518, 988.3233132138505, 1091.3762762818062, 117.13850084971784, 228.77340395785214, 554.3587196943787, 924.1931162179422, 952.9223813447796]
Q2 = 6903.691689225772
Выборочное значение статистики: 3.886705765898007
Критическая область: (1.889525776660891; +inf)
p-value: 7.334946486659e-05



У студентов измерили пульс до и после тренировки. Верно ли, что пульс в среднем увеличился на 50 ударов в минуту?

Данные (ударов в минуту)

Проверьте данное предположение с помощью доверительных интервалов. Уровень значимости 0.02

Сформулируйте нулевую и альтернативную гипотезы
Проверьте данное предположение с помощью доверительных интервалов. Уровень значимости 0.02

Постройте приближенный доверительный интервал для a1−a2
. Примите статистическое решение


In [ ]:
# prepare data
a = pd.read_csv('pulse.txt', sep='\t')


In [73]:
# задаем распределение Фишера, которое будет использоваться
import scipy as sc
f_local = sc.stats.norm()
a1 = a['Pulse1']
a2 = a['Pulse2']
N = len(a)
k = 2
alpha = 0.02

sigma1= (a1-a2).var()
print(f"sigma   -- {sigma1}")
thetha = (a1 - a2).mean()
delta = np.sqrt(sigma1)*f_local.ppf(1-alpha/2)/ np.sqrt(N)

left = thetha - delta
right = thetha + delta
Fv = (Q2 / (k - 1)) / (Q1 / (N - k))
print(f"интервал: ({left}, {right})")
print(f"решение: {(left < -50) & (right >=-50)}")

sigma   -- 444.7768115942029
интервал: (-58.625109573126394, -44.15749912252578)
решение: True
